In [1]:
import json
import os
import time

import torch

from MD.get_person_voice import get_features_from_file
from MD.main_predict import emb_from_file, get_DTDNNSS_model, get_ResNet_model, load_db, \
    create_db, predict_model
from speechnas.nas_core.global_enum import CheckpointKey
from speechnas.network.dtdnnss_searched import DtdnnssBase_v1

os.chdir(r'../')
os.chdir(r'../')

In [2]:
with open(r'models\7. DTDNNSS_01\params_DeepSpeaker_100pers_10vox_20mfcc_tripl_011.json', 'r') as f:
    model_params = json.load(f)

In [7]:
model = get_DTDNNSS_model(weights_file=r'models\7. DTDNNSS_01\DTDNNSS_20mfcc_001.pth')

In [3]:
model = get_ResNet_model()

In [3]:
model = DtdnnssBase_v1(num_class=7323,
                       mid_channels=[64, 96, 128, 128, 160],
                       reduction_channels=[32, 64, 32, 64, 32],
                       chromosome=[3, 4, 2, 2, 2, 1, 1, 3, 1, 0, 0, 3, 3, 2, 4, 4, 3, 3],
                       feature_dim=128)
state_dict = torch.load(r'speechnas\checkpoint\dtdnnss_3.1M_eer1.14.pth')

model.load_state_dict(state_dict[CheckpointKey.ModelState], strict=False)
model.eval()

DtdnnssBase_v1(
  (layer1): TDNN_(
    (conv): Conv1d_(
      (conv): Conv1d(30, 128, kernel_size=(5,), stride=(1,), padding=(2,))
      (act): Identity_()
    )
    (bn): BatchNorm1d_(
      (act): Identity_()
      (bn): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (act): PReLU(num_parameters=128)
  )
  (layer2): DTDNNSS_V1_(
    (bn1): BatchNorm1d_(
      (act): Identity_()
      (bn): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (act1): PReLU(num_parameters=128)
    (conv1): Conv1d_(
      (conv): Conv1d(128, 128, kernel_size=(1,), stride=(1,), bias=False)
      (act): Identity_()
    )
    (bn2): BatchNorm1d_(
      (act): Identity_()
      (bn): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (act2): PReLU(num_parameters=128)
    (conv2): ModuleList(
      (0): Conv1d_(
        (conv): Conv1d(128, 64, kernel_size=(3,), stride=(1,), padding=(1,), b

In [9]:
# D:\Py_Projects\neuro\output\wav_voices
create_db(model, model_params, r'output\wav_voices',
          'output/db_new.json', features_type='mfcc')

End.


In [4]:
db = load_db(r'output/db_new.json')

In [7]:
test_data = r'output\test_rec_wav.wav'
start = time.time()
emb = emb_from_file(model, test_data, model_params, features_type='spectro', with_extract=False)
end = time.time()

In [8]:
mfcc, spectro = get_features_from_file(test_data, model_params, mfcc=True, spectrogram=True)

In [17]:
start = time.time()
emb = predict_model(model, spectro, with_extract=False)
end = time.time()

In [18]:
end - start

0.13299989700317383